In [3]:
import climdata
from hydra import initialize, compose
import xclim
import datetime

In [ ]:
mirror = climdata.ERA5(base_path="./zarr_tas_data")

# split the years into train, validation, and test
years = list(range(2025, 2025+1))
# Set the variables to download for 34 var dataset
date_range = (
    datetime.date(2000, 1, 1),
    datetime.date(2000, 1, 3),
)

mirror.download("2m_temperature",date_range)

In [ ]:
with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(config_name="config",
                  overrides=[
                      "dataset=mswx",
                      "weather.parameter=tasmax",
                      "time_range.start_date=2000-06-01",
                      "time_range.end_date=2000-06-30",
                      ])
cfg.mappings[cfg.dataset].params['google_service_account'] = "/home/muduchuru/.climdata_conf/service.json"

In [14]:
mswx_ob = climdata.MSWX(cfg)
files = mswx_ob.fetch()
dset = mswx_ob.load()
dset_subset = mswx_ob.extract(point=(52,20))
dset_subset = dset_subset.compute()
dset_subset = xclim.core.units.convert_units_to(dset_subset,cfg.mappings['info'][cfg.weather.parameter].units)

df = mswx_ob.to_dataframe(dset_subset)
df = mswx_ob.format(df)

📂 0 exist, 30 missing — fetching from Drive...
⬇️ Downloading 2025181.nc ...
   → Download 100% complete
⬇️ Downloading 2025180.nc ...
   → Download 100% complete
⬇️ Downloading 2025179.nc ...
   → Download 100% complete
⬇️ Downloading 2025178.nc ...
   → Download 100% complete
⬇️ Downloading 2025177.nc ...
   → Download 100% complete
⬇️ Downloading 2025176.nc ...
   → Download 100% complete
⬇️ Downloading 2025175.nc ...
   → Download 100% complete
⬇️ Downloading 2025174.nc ...
   → Download 100% complete
⬇️ Downloading 2025173.nc ...
   → Download 100% complete
⬇️ Downloading 2025172.nc ...
   → Download 100% complete
⬇️ Downloading 2025171.nc ...
   → Download 100% complete
⬇️ Downloading 2025170.nc ...
   → Download 100% complete
⬇️ Downloading 2025169.nc ...
   → Download 100% complete
⬇️ Downloading 2025168.nc ...
   → Download 100% complete
⬇️ Downloading 2025167.nc ...
   → Download 100% complete
⬇️ Downloading 2025166.nc ...
   → Download 100% complete
⬇️ Downloading 2025165.nc

In [ ]:
# initialize
mswx_ob = climdata.MSWX(cfg)

# fetch + load
files = mswx_ob.fetch()
dset = mswx_ob.load()

dset_subset = mswx_ob.extract(box=(
    14.0, 49.0, 24.0, 55.0
    ))
dset_subset = dset_subset.compute()

# convert to dataframe
df = mswx_ob.to_dataframe(dset_subset)

# format dataframe
formatted = mswx_ob.format(df)

In [ ]:
formatted

In [ ]:
import argparse
import climdata
from hydra import initialize, compose
import pandas as pd
from climdata import CMIP

with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(
        config_name="config",
        overrides=[
            f"weather.parameter={variables[0]}",  # Just for bounds/region
            f"region={region}",
        ],
    )

cmip = climdata.CMIP(
    experiment_id = "historical",
    source_id = "MIROC6",
    table_id = "day",
    variables = ["tasmax", "tasmin", "pr"],
    region_bounds = cfg.bounds[cfg.region]
    )
cmip.fetch()      # gets file lists
cmip.load()       # loads and merges datasets
cmip.extract(point = (52,15))
ds = cmip._subset_time(cfg.time_range.start_date, cfg.time_range.end_date)
cmip.save_netcdf("cmip_loc.nc")

In [ ]:
import argparse
import climdata
from hydra import initialize, compose
import pandas as pd
from climdata import CMIP

region="europe"

with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(
        config_name="config",
        overrides=[
            f"weather.parameter={variables[0]}",  # Just for bounds/region
            f"region={region}",
        ],
    )

cmip = climdata.CMIP(
    experiment_id = "historical",
    source_id = "MIROC6",
    table_id = "day",
    variables = ["tasmax", "tasmin", "pr"],
    region_bounds = cfg.bounds[cfg.region]
    )
cmip.fetch()      # gets file lists
cmip.load()       # loads and merges datasets

cmip.extract(box = cmip.region_bounds)
ds = cmip._subset_time(cfg.time_range.start_date, cfg.time_range.end_date)
cmip.save_netcdf("cmip_box.nc")

Saved NetCDF to cmip_box.nc


In [20]:
"/".join(cmip.col_subsets[0].df.zstore[0].split('/')[4:-4]+[region])+'/'

'CMIP/MIROC/MIROC6/historical/r4i1p1f1/day/europe/'



**Usage Example:**


In [ ]:
ds = cmip.load()
ds_time = cmip.subset_time("2000-01-01", "2005-12-31")



You can combine this with `_subset()` for both spatial and temporal subsetting.  
Let me know if you want a combined function!